In [1]:
import pandas as pd
import numpy as np
import ast
from scipy import sparse
from sklearn.metrics.pairwise import pairwise_distances, cosine_distances, cosine_similarity

In [2]:
games = pd.concat([pd.read_csv('../data/steam_games_cleaned_1.csv'), pd.read_csv('../data/steam_games_cleaned_2.csv'), pd.read_csv('../data/steam_games_cleaned_3.csv')], axis=0)
print(games.shape)
games.head()

(56733, 506)


,appid,name,genre,developer,publisher,owners,average_forever,median_forever,pos_rating_pct,total_ratings,...,Web Publishing,Well-Written,Werewolves,Western,Word Game,World War I,World War II,Wrestling,Zombies,e-sports
0,10,Counter-Strike,Action,Valve,Valve,"10,000,000-20,000,000",11666,244,0.974693,189081,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.217837
1,20,Team Fortress Classic,Action,Valve,Valve,"2,000,000-5,000,000",91,18,0.857002,6105,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000
2,30,Day of Defeat,Action,Valve,Valve,"5,000,000-10,000,000",403,26,0.900055,5423,...,0.0,0.0,0.0,0.0,0.0,0.015287,0.313376,0.0,0.0,0.000000
3,40,Deathmatch Classic,Action,Valve,Valve,"5,000,000-10,000,000",33,6,0.816689,2193,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000
4,50,Half-Life: Opposing Force,Action,Gearbox Software,Valve,"5,000,000-10,000,000",322,127,0.951749,13119,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000


In [3]:
games['has_tag'] = [games.iloc[i,72:].drop(columns = ['Free to Play','Early Access']).values.max() for i in range(len(games))]
games['has_tag'].value_counts()

has_tag
1.0    48397
0.0     8336
Name: count, dtype: int64

In [4]:
# sistem rekomendasi nanti akan menggunakan rasio tag jadi kita perlu untuk mensortir tag yang kosong
games = games.loc[games['has_tag'] > 0]

search_df = games.loc[ : , ['appid','name']]
search_df['name'] = [name.lower() for name in search_df['name']]
search_df.set_index('name', inplace=True)

In [5]:
# untuk indeks kita menggunakan appid karena nanti ada beberapa game dengan nama yang sama
games.set_index('appid', inplace=True)
games.sort_index(inplace=True)
print(games.shape)
games.head()

(48397, 506)


,name,genre,developer,publisher,owners,average_forever,median_forever,pos_rating_pct,total_ratings,price,...,Well-Written,Werewolves,Western,Word Game,World War I,World War II,Wrestling,Zombies,e-sports,has_tag
appid,,,,,,,,,,,,,,,,,,,,,
10,Counter-Strike,Action,Valve,Valve,"10,000,000-20,000,000",11666,244,0.974693,189081,9.99,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.217837,1.0
20,Team Fortress Classic,Action,Valve,Valve,"2,000,000-5,000,000",91,18,0.857002,6105,4.99,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,1.0
30,Day of Defeat,Action,Valve,Valve,"5,000,000-10,000,000",403,26,0.900055,5423,4.99,...,0.0,0.0,0.0,0.0,0.015287,0.313376,0.0,0.0,0.000000,1.0
40,Deathmatch Classic,Action,Valve,Valve,"5,000,000-10,000,000",33,6,0.816689,2193,4.99,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,1.0
50,Half-Life: Opposing Force,Action,Gearbox Software,Valve,"5,000,000-10,000,000",322,127,0.951749,13119,4.99,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,1.0


In [6]:
search = 'counter-strike'.lower()
titles = search_df[search_df.index.str.contains(search)]
print(titles)

                                   appid
name                                    
counter-strike                        10
counter-strike: condition zero        80
counter-strike: condition zero       100
counter-strike: source               240
counter-strike: global offensive     730
counter-strike nexon: studio      273110


In [7]:
search2 = 'the witcher 3: wild hunt'.lower()
titles2 = search_df[search_df.index.str.contains(search2)]
print(titles2)

                           appid
name                            
the witcher 3: wild hunt  292030


In [8]:
search_df.loc[search,'appid']

np.int64(10)

In [9]:
search_df.loc[search2,'appid']

np.int64(292030)

In [10]:
%%time
recommender = games.loc[ : , '1980s' : 'e-sports'].drop(columns = ['Free to Play','Kickstarter','Early Access'])
sparse_rec = sparse.csr_matrix(recommender)
dists = pairwise_distances(sparse_rec, metric='cosine')
recommender_df = pd.DataFrame(dists, columns=recommender.index, index=recommender.index)

CPU times: total: 1min 26s
Wall time: 12min 48s


In [11]:
top_recommendations_1 = recommender_df[search_df.loc[search,'appid']].sort_values()[1:101]
top_recs_df_1 = pd.DataFrame(top_recommendations_1)
top_recs_df_1['name'] = [games.loc[ind]['name'] for ind in top_recs_df_1.index]
top_recs_df_1[['name',top_recommendations_1.name]].rename(columns={'name':'Game Name', top_recommendations_1.name : f'Similarity to {search} (lower is better)'})

,Game Name,Similarity to counter-strike (lower is better)
appid,,
80,Counter-Strike: Condition Zero,0.054881
100,Counter-Strike: Condition Zero,0.055738
360,Half-Life Deathmatch: Source,0.111348
240,Counter-Strike: Source,0.126721
20,Team Fortress Classic,0.128983
...,...,...
209870,Blacklight: Retribution,0.234581
706990,BLOCKPOST,0.234716
677620,Splitgate: Arena Warfare,0.235451


In [12]:
top_recommendations1 = recommender_df[search_df.loc[search,'appid']].sort_values()[1:21]
top_recs_df1 = pd.DataFrame(top_recommendations1)
top_recs_df1[[top_recommendations1.name]].rename(columns={top_recommendations1.name : f'Similarity to {search} (lower is better)'})

,Similarity to counter-strike (lower is better)
appid,
80,0.054881
100,0.055738
360,0.111348
240,0.126721
20,0.128983
2350,0.136147
320,0.147429
7940,0.148882
239660,0.149587


In [13]:
print(sparse_rec)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 532472 stored elements and shape (48397, 427)>
  Coords	Values
  (0, 0)	0.0476633773971327
  (0, 1)	0.2191398249860361
  (0, 17)	1.0
  (0, 35)	0.041333085086576
  (0, 78)	0.5121951219512195
  (0, 90)	0.2951033327127164
  (0, 137)	0.8927574008564513
  (0, 147)	0.3142803947123441
  (0, 223)	0.1158071122695959
  (0, 235)	0.6248370880655372
  (0, 254)	0.138521690560417
  (0, 289)	0.1603053435114503
  (0, 320)	0.0530627443679017
  (0, 324)	0.6183206106870229
  (0, 355)	0.1130143362502327
  (0, 362)	0.0547384099795196
  (0, 366)	0.2453919195680506
  (0, 369)	0.3423943399739341
  (0, 409)	0.0117296592813256
  (0, 426)	0.2178365295103332
  (1, 1)	0.1758389261744966
  (1, 17)	1.0
  (1, 77)	0.2429530201342281
  (1, 78)	0.3114093959731543
  (1, 80)	0.1194630872483221
  :	:
  (48394, 280)	0.4305555555555556
  (48394, 329)	0.2083333333333333
  (48395, 31)	0.59375
  (48395, 68)	1.0
  (48395, 185)	0.328125
  (48395, 226)	0.484375
  (48395,

In [14]:
print(dists)

[[0.         0.12898338 0.26106445 ... 1.         0.97928557 0.80375129]
 [0.12898338 0.         0.42419027 ... 1.         1.         0.74059917]
 [0.26106445 0.42419027 0.         ... 0.99573329 0.98870585 0.92475151]
 ...
 [1.         1.         0.99573329 ... 0.         0.94108244 0.53059439]
 [0.97928557 1.         0.98870585 ... 0.94108244 0.         0.97791333]
 [0.80375129 0.74059917 0.92475151 ... 0.53059439 0.97791333 0.        ]]


In [15]:
print(recommender_df)

appid     10        20        30        40        50        60        70       \
appid                                                                           
10       0.000000  0.128983  0.261064  0.224437  0.278565  0.244869  0.445805   
20       0.128983  0.000000  0.424190  0.113554  0.403550  0.130075  0.516827   
30       0.261064  0.424190  0.000000  0.575179  0.175826  0.585357  0.561589   
40       0.224437  0.113554  0.575179  0.000000  0.472430  0.015102  0.587937   
50       0.278565  0.403550  0.175826  0.472430  0.000000  0.480382  0.338486   
...           ...       ...       ...       ...       ...       ...       ...   
1811140  1.000000  1.000000  1.000000  1.000000  0.992233  0.989253  1.000000   
1811290  1.000000  1.000000  0.995528  1.000000  0.967347  1.000000  0.944031   
1813890  1.000000  1.000000  0.995733  1.000000  0.922362  0.977489  0.856147   
1814100  0.979286  1.000000  0.988706  1.000000  0.954813  1.000000  0.950211   
1814590  0.803751  0.740599 

In [22]:
#melihat kekosongan matriks
m, n = sparse_rec.shape
matrix_size = m*n

num_purchases = len(sparse_rec.nonzero()[0])
sparsity = 100*(1 - (num_purchases/matrix_size ))
print(sparsity)

97.42337949509034
